In [1]:
#  ******Importing Training Data******
import pandas as pd
df_initial = pd.read_csv("Dataset_disease.csv")
df_initial.head()

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
4,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection


In [2]:
df=df_initial.replace({'prognosis':{'(vertigo) Paroymsal  Positional Vertigo':0,'Acne':1,'AIDS':2,'Alcoholic hepatitis':3,'Allergy':4,
'Arthritis':5,'Bronchial Asthma':6,'Cervical spondylosis':7,'Chicken pox':8,'Chronic cholestasis':9,'Common Cold':10,
'Dengue':11,'Diabetes ':12,'Dimorphic hemmorhoids(piles)':13,'Drug Reaction':14,'Fungal infection':15,'Gastroenteritis':16,
'GERD':17,'Heart attack':18,'hepatitis A':19,'Hepatitis B':20,'Hepatitis C':21,'Hepatitis D':22,'Hepatitis E':23,
'Hypertension ':24,'Hyperthyroidism':25,'Hypoglycemia':26,'Hypothyroidism':27,'Impetigo':28,'Jaundice':29,'Malaria':30,
'Migraine':31,'Osteoarthristis':32,'Paralysis (brain hemorrhage)':33,'Peptic ulcer diseae':34,'Pneumonia':35,'Psoriasis':36,
'Tuberculosis':37,'Typhoid':38,'Urinary tract infection':39,'Varicose veins':40}})

In [3]:
#  ****Combining 132 features of the Training data into group of 12 features****
def column_grouping(a,b,c,d,e,f,g,h,i,j,k,l):
    return (2**0*a+2**1*b+2**2*c+2**3*d+2**4*e+2**5*f+2**6*g+2**7*h+2**8*i+2**9*j+2**10*k+2**11*l)
list1 = []
for i in range(len(df.index)):
    l = []
    j = 0
    while j < 132:
        f = column_grouping(df.loc[i][j],df.loc[i][j+1],df.loc[i][j+2],df.loc[i][j+3],df.loc[i][j+4],df.loc[i][j+5],df.loc[i][j+6],df.loc[i][j+7],df.loc[i][j+8],df.loc[i][j+9],df.loc[i][j+10],df.loc[i][j+11])
        l.append(f)
        j+=12
    l.append(df.loc[i][132])
    list1.append(l)

In [4]:
#  ***Creating new dataset csv file for the combined features****
df = pd.DataFrame(list1,columns = ['A','B','C','D','E','F','G','H','I','J','K','prognosis'])
df.to_csv("Dataset_new1.csv",index = False, header = True)

In [5]:
#  ******Importing the new training data******
df = pd.read_csv("Dataset_new.csv")
X = df.drop('prognosis', axis = 'columns')
y = df['prognosis']

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
X_train

,A,B,C,D,E,F,G,H,I,J,K
2644,1,516,2816,200,1,0,0,0,0,24,0
2881,256,0,192,0,2,0,516,2048,1,4,0
537,0,0,0,2,512,24,0,2,0,0,0
2466,1024,1024,2,0,0,0,8,0,0,0,0
300,0,4,0,0,0,992,0,0,0,2048,0
...,...,...,...,...,...,...,...,...,...,...,...
4344,66,0,0,0,0,0,0,0,0,0,480
444,2049,0,2304,136,0,0,0,0,0,0,0
4008,0,620,0,0,0,3088,3,2048,33,0,0
4134,64,0,0,0,0,8,1216,0,0,0,2


In [7]:
#  *****using SVM Classifier*****
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
svm_model = SVC(C = 1, gamma = 1, kernel = 'linear',probability=True)
svm_model.fit(X_train,y_train)
y_pred_svm = svm_model.predict(X_test)
print(accuracy_score(y_test, y_pred_svm))

0.9969788519637462


In [8]:
#  ***** Using Cross Validation *****
import numpy as np
from sklearn.model_selection import cross_val_score
def score_cv(model, X, y):
    score = cross_val_score(model, X, y, cv=5)
    print(score)
    return np.mean(score)

In [9]:
svm_score = score_cv(svm_model, X, y)
svm_score

[0.9979859  0.99697885 1.         1.         0.99899194]


0.9987913377513562

In [10]:
#  ***** using GridSearchCV on SVM ******
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(SVC(gamma = 'auto'),{
    'C':[1,3,5,7,10],
    'kernel':['linear', 'poly', 'rbf']
}, cv = 5, return_train_score = False)

clf.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=SVC(gamma='auto'),
             param_grid={'C': [1, 3, 5, 7, 10],
                         'kernel': ['linear', 'poly', 'rbf']})

In [11]:
df2 = pd.DataFrame(clf.cv_results_)
df2[['param_C', 'param_kernel', 'mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,linear,0.997480
1,1,poly,0.999244
2,1,rbf,0.997480
3,3,linear,0.998236
4,3,poly,0.999244
5,3,rbf,0.998236
6,5,linear,0.999244
7,5,poly,0.999244
8,5,rbf,0.998236
9,7,linear,0.999244


In [12]:
clf.best_params_

{'C': 1, 'kernel': 'poly'}

In [13]:
from tkinter import *
from tkinter import messagebox
import numpy as np
import pandas as pd
from PIL import ImageTk,Image

In [14]:
l1=['itching','skin_rash','nodal_skin_eruptions','continuous_sneezing','shivering','chills','joint_pain',
    'stomach_pain','acidity','ulcers_on_tongue','muscle_wasting','vomiting','burning_micturition','spotting_ urination','fatigue',
    'weight_gain','anxiety','cold_hands_and_feets','mood_swings','weight_loss','restlessness','lethargy','patches_in_throat',
    'irregular_sugar_level','cough','high_fever','sunken_eyes','breathlessness','sweating','dehydration','indigestion',
    'headache','yellowish_skin','dark_urine','nausea','loss_of_appetite','pain_behind_the_eyes','back_pain','constipation',
    'abdominal_pain','diarrhoea','mild_fever','yellow_urine','yellowing_of_eyes','acute_liver_failure','fluid_overload',
    'swelling_of_stomach','swelled_lymph_nodes','malaise','blurred_and_distorted_vision','phlegm','throat_irritation',
    'redness_of_eyes','sinus_pressure','runny_nose','congestion','chest_pain','weakness_in_limbs','fast_heart_rate',
    'pain_during_bowel_movements','pain_in_anal_region','bloody_stool','irritation_in_anus','neck_pain','dizziness','cramps',
    'bruising','obesity','swollen_legs','swollen_blood_vessels','puffy_face_and_eyes','enlarged_thyroid','brittle_nails',
    'swollen_extremeties','excessive_hunger','extra_marital_contacts','drying_and_tingling_lips','slurred_speech','knee_pain','hip_joint_pain',
    'muscle_weakness','stiff_neck','swelling_joints','movement_stiffness','spinning_movements','loss_of_balance','unsteadiness','weakness_of_one_body_side',
    'loss_of_smell','bladder_discomfort','foul_smell_of urine','continuous_feel_of_urine','passage_of_gases','internal_itching','toxic_look_(typhos)',
    'depression','irritability','muscle_pain','altered_sensorium','red_spots_over_body','belly_pain','abnormal_menstruation','dischromic _patches',
    'watering_from_eyes','increased_appetite','polyuria','family_history','mucoid_sputum','rusty_sputum','lack_of_concentration','visual_disturbances',
    'receiving_blood_transfusion','receiving_unsterile_injections','coma','stomach_bleeding','distention_of_abdomen','history_of_alcohol_consumption',
    'fluid_overload','blood_in_sputum','prominent_veins_on_calf','palpitations','painful_walking','pus_filled_pimples','blackheads','scurring','skin_peeling',
    'silver_like_dusting','small_dents_in_nails','inflammatory_nails','blister','red_sore_around_nose','yellow_crust_ooze']
l2=[]
for x in range(0,len(l1)):
    l2.append(0)


In [15]:
my_dict={'(vertigo) Paroymsal  Positional Vertigo':0,'Acne':1,'AIDS':2,'Alcoholic hepatitis':3,'Allergy':4,
'Arthritis':5,'Bronchial Asthma':6,'Cervical spondylosis':7,'Chicken pox':8,'Chronic cholestasis':9,'Common Cold':10,
'Dengue':11,'Diabetes ':12,'Dimorphic hemmorhoids(piles)':13,'Drug Reaction':14,'Fungal infection':15,'Gastroenteritis':16,
'GERD':17,'Heart attack':18,'hepatitis A':19,'Hepatitis B':20,'Hepatitis C':21,'Hepatitis D':22,'Hepatitis E':23,
'Hypertension ':24,'Hyperthyroidism':25,'Hypoglycemia':26,'Hypothyroidism':27,'Impetigo':28,'Jaundice':29,'Malaria':30,
'Migraine':31,'Osteoarthristis':32,'Paralysis (brain hemorrhage)':33,'Peptic ulcer diseae':34,'Pneumonia':35,'Psoriasis':36,
'Tuberculosis':37,'Typhoid':38,'Urinary tract infection':39,'Varicose veins':40}
key_list = list(my_dict.keys())
val_list = list(my_dict.values())


In [16]:
def message():
    if (Symptom1.get() == "None" or Symptom2.get() == "None" or Symptom3.get() == "None" or Symptom4.get() == "None" or Symptom5.get() == "None"):
        messagebox.showinfo("OPPS!!", "ENTER  SYMPTOMS PLEASE")
    else :
        SVM()

def SVM():
    psymptoms = [Symptom1.get(),Symptom2.get(),Symptom3.get(),Symptom4.get(),Symptom5.get()]
    for k in range(0,len(l1)):
        for z in psymptoms:
            if(z==l1[k]):
                l2[k]=1
    #  ****Combining 132 features of the Training data into group of 12 features****
    def column_grouping(a,b,c,d,e,f,g,h,i,j,k,l):
        return (2**0*a+2**1*b+2**2*c+2**3*d+2**4*e+2**5*f+2**6*g+2**7*h+2**8*i+2**9*j+2**10*k+2**11*l)
    list1 = []
    j=0
    while j < 132:
        f = column_grouping(l2[j],l2[j+1],l2[j+2],l2[j+3],l2[j+4],l2[j+5],l2[j+6],l2[j+7],l2[j+8],l2[j+9],l2[j+10],l2[j+11])
        list1.append(f)
        j+=12 
    y_pred = svm_model.predict([list1])    
    predicted=y_pred[0]
    position = val_list.index(predicted)
    print(key_list[position])
    h='yes'
    if (h=='yes'):
        t3.delete("1.0", END)
        t3.insert(END, key_list[position])
    else:
        t3.delete("1.0", END)
        t3.insert(END, "No Disease")

In [21]:
root = Tk()
root.title(" Disease Prediction From Symptoms")
root.configure(background="Lightcyan2")
bg_image = PhotoImage(file ="da2.png")
x = Label (image = bg_image)
x.grid(row = 0, column = 0)
bg1_image = PhotoImage(file ="da3.png")
y = Label (image = bg1_image)
y.grid(row = 0, column = 2,padx=200)
Symptom1 = StringVar()
Symptom1.set(None)
Symptom2 = StringVar()
Symptom2.set(None)
Symptom3 = StringVar()
Symptom3.set(None)
Symptom4 = StringVar()
Symptom4.set(None)
Symptom5 = StringVar()
Symptom5.set(None)

w2 = Label(root, justify=LEFT, text=" Disease Prediction From Symptoms " , fg="Red", bg="Ivory")
w2.config(font=("Elephant", 30))
w2.grid(row=0, column=1, columnspan=1, padx=20)

NameLb1 = Label(root, text="ML MODEL : SVM")
NameLb1.config(font=("Times",20))
NameLb1.grid(row=1, column=1, pady=20, padx=250, sticky=W)

S1Lb = Label(root,  text="Symptom 1")
S1Lb.config(font=("Elephant", 15))
S1Lb.grid(row=7, column=1, pady=10 , sticky=W)

S2Lb = Label(root,  text="Symptom 2")
S2Lb.config(font=("Elephant", 15))
S2Lb.grid(row=8, column=1, pady=10, sticky=W)

S3Lb = Label(root,  text="Symptom 3")
S3Lb.config(font=("Elephant", 15))
S3Lb.grid(row=9, column=1, pady=10, sticky=W)

S4Lb = Label(root,  text="Symptom 4")
S4Lb.config(font=("Elephant", 15))
S4Lb.grid(row=10, column=1, pady=10, sticky=W)

S5Lb = Label(root,  text="Symptom 5")
S5Lb.config(font=("Elephant", 15))
S5Lb.grid(row=11, column=1, pady=10, sticky=W)

lr = Button(root, text="Predict",height=2,fg="White", bg="green", width=20, command=message)
lr.config(font=("Elephant", 15))
lr.grid(row=12, column=1,pady=20)

OPTIONS = sorted(l1)

S1En = OptionMenu(root, Symptom1,*OPTIONS)
S1En.grid(row=7, column=2)

S2En = OptionMenu(root, Symptom2,*OPTIONS)
S2En.grid(row=8, column=2)

S3En = OptionMenu(root, Symptom3,*OPTIONS)
S3En.grid(row=9, column=2)

S4En = OptionMenu(root, Symptom4,*OPTIONS)
S4En.grid(row=10, column=2)

S5En = OptionMenu(root, Symptom5,*OPTIONS)
S5En.grid(row=11, column=2)

NameLb = Label(root, text="")
NameLb.config(font=("Elephant", 20))
NameLb.grid(row=12, column=1, pady=10,  sticky=W)


t3 = Text(root, height=2, width=30)
t3.config(font=("Elephant", 20) , bg="thistle2")
t3.grid(row=14, column=1 , padx=10)

root.mainloop()

Alcoholic hepatitis
